<a href="https://colab.research.google.com/github/eugeneteoh/ai-algorithms/blob/transformer/transformer/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference:

https://github.com/Atcold/pytorch-Deep-Learning/blob/master/15-transformer.ipynb

https://nn.labml.ai/transformers

In [ ]:
%pip install torchdata torchtext pytorch-lightning

In [99]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import numpy as np


In [110]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [111]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model=512, num_heads=8):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = self.d_v = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model, bias=False)
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)

        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V):
        A = Q @ K.transpose(2, 3)
        A /= np.sqrt(self.d_k)
        A = F.softmax(A, dim=-1)
        attn = A @ V
        return attn

    def forward(self, X_q, X_k, X_v):
        batch_size, seq_length, dim = X_q.shape

        Q = self.W_q(X_q)
        K = self.W_k(X_k)
        V = self.W_v(X_v)

        # Split heads
        Q = Q.view(batch_size, self.num_heads, seq_length, self.d_k)
        K = K.view(batch_size, self.num_heads, seq_length, self.d_k)
        V = V.view(batch_size, self.num_heads, seq_length, self.d_v)

        H_cat = self.scaled_dot_product_attention(Q, K, V)
        H_cat = H_cat.view(batch_size, seq_length, dim)

        out = self.W_o(H_cat)
        return out

mha = MultiHeadAttention()

In [112]:
test_K = torch.tensor(
    [[10, 0, 0],
     [ 0,10, 0],
     [ 0, 0,10],
     [ 0, 0,10]]
).float()[None,None]

test_V = torch.tensor(
    [[   1,0,0],
     [  10,0,0],
     [ 100,5,0],
     [1000,6,0]]
).float()[None,None]

test_Q = torch.tensor(
    [[0, 10, 0]]
).float()[None, None]

test_X_k = torch.randn((1, 1, 512))
test_X_v = torch.randn((1, 1, 512))
test_X_q = torch.randn((1, 1, 512))

# Test scaled_dot_product_attention shape
output = mha.scaled_dot_product_attention(test_Q, test_K, test_V)
assert test_Q.shape == output.shape

# Test mha output shape
output = mha(test_X_q, test_X_k, test_X_v)
assert test_X_q.shape == output.shape

In [113]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model=512, num_heads=8, ff_hidden_dim=2048):
        super().__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ff = nn.Sequential(
            nn.Linear(d_model, ff_hidden_dim),
            nn.ReLU(),
            nn.Linear(ff_hidden_dim, d_model)
        )

        self.layernorm1 = nn.LayerNorm(normalized_shape=d_model)
        self.layernorm2 = nn.LayerNorm(normalized_shape=d_model)

    def forward(self, x):
        out = self.layernorm1(x + self.mha(x, x, x))
        out = self.layernorm2(out + self.ff(out))
        return out

In [114]:
class Embedding(nn.Module):
    def __init__(self, d_model=512, vocab_size=10000, max_len=5000):
        super().__init__()

        self.word_embeddings = nn.Embedding(vocab_size, d_model, padding_idx=1)
        self.register_buffer("positional_encodings", self.get_positional_encoding(d_model, max_len))
        self.layernorm = nn.LayerNorm(d_model)

    def get_positional_encoding(self, d_model, max_len):
        encodings = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        two_i = torch.arange(0, d_model, 2)
        denominator = 10000 ** (two_i / d_model)
        div = position / denominator
        encodings[:, 0::2] = torch.sin(div)
        encodings[:, 1::2] = torch.cos(div)
        encodings.requires_grad_(False)

        return encodings

    def forward(self, x):
        seq_length = x.shape[1]
        position_ids = torch.arange(seq_length, dtype=torch.long, device=x.device) # (max_seq_length)
        position_ids = position_ids.unsqueeze(0).expand_as(x)                      # (bs, max_seq_length)

        word_embeddings = self.word_embeddings(x)

        embeddings = word_embeddings + self.positional_encodings
        
        return self.layernorm(embeddings)


In [115]:
class Encoder(nn.Module):
    def __init__(self, vocab_size=10000, max_seq_len=5000, num_layers=6, d_model=512, num_heads=8, ff_hidden_dim=2048):
        super().__init__()

        self.embedding_layer = Embedding(d_model=d_model, vocab_size=vocab_size, max_len=max_seq_len)
        self.enc_layers = nn.Sequential(*[EncoderLayer(d_model=d_model, num_heads=num_heads, ff_hidden_dim=ff_hidden_dim) for _ in range(num_layers)])

    def forward(self, x):
        x = self.embedding_layer(x)
        return self.enc_layers(x)


In [116]:
class TransformerClassifier(nn.Module):
    def __init__(self, num_outputs, vocab_size=10000, max_seq_len=5000, num_layers=6, d_model=512, num_heads=8, ff_hidden_dim=2048):
        super().__init__()

        self.encoder = Encoder(
            vocab_size=vocab_size, max_seq_len=max_seq_len, num_layers=num_layers, d_model=d_model, num_heads=num_heads, ff_hidden_dim=ff_hidden_dim
        )
        self.dense = nn.Linear(d_model, num_outputs)

    def forward(self, x):
        x = self.encoder(x)
        x, _ = torch.max(x, dim=1)
        x = self.dense(x)
        return x



In [117]:
from torchtext.datasets import AG_NEWS, IMDB
from collections import Counter
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import vocab
from torchtext.functional import truncate, to_tensor

from torch.utils.data import DataLoader
import torchtext.transforms as T


In [118]:
tokenizer = get_tokenizer('basic_english')  
train_iter = IMDB(split='train')
test_iter = IMDB(split='test')
counter = Counter()
for (label, line) in train_iter:
    counter.update(tokenizer(line))
data_vocab = vocab(counter, min_freq = 1, specials=('\<unk\>', '\<BOS\>', '\<EOS\>', '\<PAD\>'))

batch_size = 64
max_seq_len = 256

text_transform = T.Sequential(
    T.VocabTransform(data_vocab),
    T.Truncate(max_seq_len),
    T.ToTensor(),
    T.PadTransform(max_seq_len, 1),
)
text_pipeline = lambda x: text_transform(tokenizer(x))
label_pipeline = lambda x: int(x) - 1
apply_transform = lambda x: (label_pipeline(x[0]), text_pipeline(x[1]))

train_iter = train_iter.map(apply_transform)
train_iter = train_iter.batch(batch_size)
train_iter = train_iter.rows2columnar(["target", "token_ids"])
train_loader = DataLoader(train_iter, batch_size=None)

In [119]:
classifier = TransformerClassifier(num_outputs=2, vocab_size=len(data_vocab), max_seq_len=max_seq_len)

for i, batch in enumerate(train_loader):
    targets = batch["target"]
    token_ids = torch.stack(batch["token_ids"])

    out = classifier(token_ids)
    print(out.shape)
    if i == 0:
        break

torch.Size([64, 2])


In [120]:
import pytorch_lightning as pl

In [121]:
class TransformerClassifierLT(pl.LightningModule):
    def __init__(self, num_outputs, vocab_size=10000, max_seq_len=5000, num_layers=6, d_model=512, num_heads=8, ff_hidden_dim=2048):
        super().__init__()
        self.classifier = TransformerClassifier(
            num_outputs=num_outputs,
            vocab_size=vocab_size,
            max_seq_len=max_seq_len,
            num_layers=num_layers,
            d_model=d_model,
            num_heads=num_heads,
            ff_hidden_dim=ff_hidden_dim
        )

    def training_step(self, batch, batch_idx):
        targets = torch.as_tensor(batch["target"])
        token_ids = torch.stack(batch["token_ids"])

        out = self.classifier(token_ids)

        loss = F.cross_entropy(out, targets)

        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

classifier = TransformerClassifierLT(num_outputs=2, vocab_size=len(data_vocab), max_seq_len=max_seq_len, num_layers=2, d_model=32, num_heads=4)

In [122]:
trainer = pl.Trainer(limit_train_batches=100, max_epochs=1)
trainer.fit(model=classifier, train_dataloaders=train_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                  | Params
-----------------------------------------------------
0 | classifier | TransformerClassifier | 3.5 M 
-----------------------------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params
13.988    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
